In [4]:
import numpy as np
import math
import torch
from torch import nn

import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt

from tempfile import TemporaryDirectory

from Models import BigGenerator
from Tools import KL, batchKL, sw, FunSW, FunKL

from tqdm import trange

from Experiments import get_setup

import itertools

from pandas import DataFrame
import pandas as pd

print(torch.__version__)


1.4.0


In [33]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [32]:
lat_dim=5

In [2]:
models={'foong':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'foong_sparse':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'foong_mixed':{'HMC':None,
                  'GeNNeVI':None,
                  'FuNNeVI':None},
        'boston':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'concrete':{'HMC':None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
        'energy':{'HMC':None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'powerplant':{'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'wine':  {'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None},
       'yacht': {'HMC': None,
                 'GeNNeVI':None,
                 'FuNNeVI':None}}

In [47]:
datasets=[d for d,i in models.items()]
datasets_foong=['foong', 'foong_sparse', 'foong_mixed']
datasets

['foong',
 'foong_sparse',
 'foong_mixed',
 'boston',
 'concrete',
 'energy',
 'powerplant',
 'wine',
 'yacht']

In [11]:
models_HMC=torch.load('Results/models_HMC.pt')
for d in datasets:
    models[d]['HMC']=[models_HMC[d]]
    

In [17]:
FuNmodelsFoong=torch.load('Results/FuNmodelsFoong.pt')

In [18]:
FuNmodels=torch.load('Results/FuNmodels.pt')
for d in datasets_foong:
    FuNmodels[d]=FuNmodelsFoong[d]

In [21]:
torch.save(FuNmodels,'Results/FuNmodels.pt')

In [23]:
for d in datasets:
    models[d]['FuNNeVI']=FuNmodels[d]

In [27]:
results_f=torch.load('Results/results_new_bis.pt')
pd.DataFrame(results_f, index=datasets)

,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(93.609, 3.109)","(71.562, 15.695)","(93.524, 4.952)","(61.826, 20.352)","(0.056, 0.001)","(0.078, 0.019)","(48.479, 17.907)","(45.12, 22.522)","(1.022, 0.028)","(1.191, 0.14)"
foong_sparse,"(74.764, 3.025)","(45.143, 1.499)","(40.717, 4.308)","(50.954, 5.457)","(0.019, 0.002)","(0.017, 0.003)","(39.101, 3.057)","(47.446, 14.567)","(0.841, 0.062)","(0.754, 0.026)"
foong_mixed,NaN,"(48.198, 3.087)",NaN,"(49.068, 6.669)",NaN,"(0.012, 0.001)",NaN,"(31.927, 2.951)",NaN,"(0.86, 0.003)"
boston,"(66.876, 2.614)","(3.661, 0.471)","(9.968, 0.375)","(66.12, 0.482)","(0.427, 0.002)","(1.543, 0.024)","(12.405, 3.185)","(2.973, 0.489)","(0.314, 0.003)","(3.152, 0.226)"
concrete,"(61.895, 1.988)","(2.967, 0.313)","(5.907, 0.21)","(61.125, 1.301)","(0.315, 0.004)","(0.963, 0.028)","(3.585, 0.841)","(2.861, 1.133)","(0.306, 0.016)","(3.665, 0.1)"
energy,"(44.847, 12.241)","(4.747, 0.443)","(12.987, 3.607)","(63.431, 0.583)","(1.077, 1.059)","(0.552, 0.006)","(34.873, 9.787)","(3.769, 0.457)","(0.408, 0.017)","(3.129, 0.038)"
powerplant,"(99.755, 1.941)","(10.413, 1.031)","(9.343, 0.337)","(84.998, 2.472)","(0.108, 0.0)","(0.604, 0.016)","(26.304, 13.344)","(7.367, 1.449)","(0.284, 0.002)","(4.619, 0.087)"
wine,"(79.141, 5.465)","(56.156, 4.475)","(20.131, 3.774)","(42.665, 0.464)","(0.623, 0.077)","(1.769, 0.059)","(63.66, 5.092)","(28.65, 3.819)","(0.848, 0.026)","(0.967, 0.057)"
yacht,"(97.657, 4.306)","(7.737, 1.614)","(14.405, 1.089)","(65.408, 2.995)","(0.238, 0.001)","(0.533, 0.027)","(13.611, 1.388)","(4.921, 2.939)","(0.313, 0.007)","(4.433, 0.103)"


In [52]:
torch.save(models, "Results/The_models.pt")

In [28]:
metrics=[m for m,d in results_f.items()]
metrics

['KL(G,HMC)',
 'KL(F,HMC)',
 'KL(HMC,G)',
 'KL(HMC,F)',
 'SW(HMC,G)',
 'SW(HMC,F)',
 'KL(G,G)',
 'KL(F,F)',
 'SW(G,G)',
 'SW(F,F)']

In [37]:
datasets=datasets_foong

In [38]:
metric='KL(F,HMC)'
for dataset in datasets:

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())

    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 54.94859313964844
foong: 53.53559112548828
foong: 53.171634674072266
foong_sparse: 40.3542366027832
foong_sparse: 55.23053741455078
foong_sparse: 47.725990295410156
foong_mixed: 53.08280944824219
foong_mixed: 57.857879638671875
foong_mixed: 43.379451751708984


In [39]:
metric='KL(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)

    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
            K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())

    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 43.25922775268555
foong: 34.87715530395508
foong: 39.90632247924805
foong_sparse: 41.45333480834961
foong_sparse: 50.55454635620117
foong_sparse: 43.761695861816406
foong_mixed: 59.61448287963867
foong_mixed: 65.65032958984375
foong_mixed: 43.787601470947266


In [40]:
metric='SW(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))



foong: 0.06357075273990631
foong: 0.0486682653427124
foong: 0.060762859880924225
foong_sparse: 0.017405826598405838
foong_sparse: 0.028838658705353737
foong_sparse: 0.023587116971611977
foong_mixed: 0.013164439238607883
foong_mixed: 0.016316985711455345
foong_mixed: 0.00915541872382164


In [41]:
metric='KL(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: (25.953933715820312, 26.312511444091797)
foong: (18.016586303710938, 18.546812057495117)
foong: (17.462556838989258, 17.998811721801758)
foong_sparse: (29.38646697998047, 26.8703556060791)
foong_sparse: (20.645307540893555, 20.444438934326172)
foong_sparse: (18.565528869628906, 18.93509292602539)
foong_mixed: (24.31157875061035, 28.666982650756836)
foong_mixed: (40.3663444519043, 35.92378234863281)
foong_mixed: (47.655818939208984, 35.288307189941406)


In [42]:
metric='SW(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))


0.7202625842285777
0.7600973131923526
0.8162232281721759
0.8392144144502784
0.7753943617308598
0.8675571039706714
0.7407402978842316
0.8828219287238174
0.9445834493885477


In [51]:
torch.save(results_f, 'Results/The_results-FuS.pt')
pd.DataFrame(results_f,index=datasets)


,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(93.609, 3.109)","(53.885, 0.766)","(93.524, 4.952)","(39.348, 3.445)","(0.056, 0.001)","(0.058, 0.006)","(48.479, 17.907)","(20.715, 3.845)","(1.022, 0.028)","(0.766, 0.039)"
foong_sparse,"(74.764, 3.025)","(47.77, 6.073)","(40.717, 4.308)","(45.257, 3.863)","(0.019, 0.002)","(0.023, 0.005)","(39.101, 3.057)","(22.475, 4.131)","(0.841, 0.062)","(0.827, 0.039)"
foong_mixed,NaN,"(51.44, 6.024)",NaN,"(56.351, 9.219)",NaN,"(0.013, 0.003)",NaN,"(35.369, 7.566)",NaN,"(0.856, 0.085)"
boston,"(66.876, 2.614)","(3.713, 0.457)","(9.968, 0.375)","(66.41, 0.321)","(0.427, 0.002)","(1.543, 0.024)","(12.405, 3.185)","(2.973, 0.489)","(0.314, 0.003)","(3.152, 0.226)"
concrete,"(61.895, 1.988)","(3.092, 0.396)","(5.907, 0.21)","(61.223, 1.48)","(0.315, 0.004)","(0.963, 0.028)","(3.585, 0.841)","(2.861, 1.133)","(0.306, 0.016)","(3.665, 0.1)"
energy,"(44.847, 12.241)","(4.666, 0.39)","(12.987, 3.607)","(63.492, 0.493)","(1.077, 1.059)","(0.552, 0.006)","(34.873, 9.787)","(3.769, 0.457)","(0.408, 0.017)","(3.129, 0.038)"
powerplant,"(99.755, 1.941)","(10.611, 0.845)","(9.343, 0.337)","(84.8, 2.366)","(0.108, 0.0)","(0.604, 0.016)","(26.304, 13.344)","(7.367, 1.449)","(0.284, 0.002)","(4.619, 0.087)"
wine,"(79.141, 5.465)","(56.074, 4.187)","(20.131, 3.774)","(42.665, 0.464)","(0.623, 0.077)","(1.769, 0.059)","(63.66, 5.092)","(28.65, 3.819)","(0.848, 0.026)","(0.967, 0.057)"
yacht,"(97.657, 4.306)","(7.647, 1.742)","(14.405, 1.089)","(65.408, 2.995)","(0.238, 0.001)","(0.533, 0.027)","(13.611, 1.388)","(4.921, 2.939)","(0.313, 0.007)","(4.433, 0.103)"
